In [3]:
## KAGGLE ONLY
# from shutil import copyfile
# copyfile(src="../input/inputs/generate_dataloaders.py", dst="../working/generate_dataloaders.py")
# copyfile(src="../input/inputs/train_dataloader.p", dst="../working/train_dataloader.p")
# copyfile(src="../input/inputs/val_dataloader.p", dst="../working/val_dataloader.p")
# copyfile(src="../input/inputs/centroids_dataloader.p", dst="../working/ground_truth_dataloader.p")
# copyfile(src="../input/inputs/dictionary.p", dst="../working/dictionary.p")

In [33]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

#from datasets import get_mnist_dataset, get_data_loader
#from utils import *
#from models import *

import pickle as pkl
import os
import datetime as dt
import pandas as pd
import random

from generate_dataloaders import *

from tqdm import tqdm_notebook as tqdm

## Get Dataloaders

In [35]:
seed = 1029
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def _init_fn(worker_id):
    np.random.seed(int(seed))

In [36]:
path = os.getcwd()
data_dir = path + '/'
data_dir = path +'/data/' #Uncomment for local system

#### *Verify filenames are consistent*

In [37]:
#train_loader = pkl.load(open(data_dir + 'train_dataloader.p','rb'))
train_loader_labelled = pkl.load(open(data_dir + 'train_labelled_dataloader.p','rb'))
train_loader_unlabelled = pkl.load(open(data_dir + 'train_unlabelled_dataloader.p','rb'))
val_loader = pkl.load(open(data_dir + 'val_dataloader.p','rb'))
#centroids = pkl.load(open(data_dir + 'train_labeld_dataloader.p','rb'))

In [38]:
review_dict = pkl.load(open(data_dir + 'dictionary.p','rb'))

In [39]:
#%conda install pytorch torchvision -c pytorch
## if torch.__version__ is not 1.3.1, run this cell then restart kernel

In [40]:
print(torch.__version__)

1.3.1


## PRE TRAINED WORD EMBEDDINGS 

In [42]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float16')

In [43]:
def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

In [44]:
def build_matrix(review_dict, embedding_index ,dim = 200):
#     embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(review_dict.tokens), dim))
    unknown_words = []
    
    for word, i in review_dict.ids.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

In [45]:
#glove_twitter = '../input/glove-global-vectors-for-word-representation/glove.twitter.27B.200d.txt' #Change loc for local system
glove_twitter = data_dir + 'glove.twitter.27B.200d.txt'

In [46]:
embedding_index = load_embeddings(glove_twitter)

In [47]:
glove_embedding_index,unknown_words = build_matrix(review_dict, embedding_index)
del embedding_index

In [48]:
len(review_dict.tokens)

16256

In [49]:
len(unknown_words)

4428

In [50]:
for word in unknown_words:
    print(word)

<pad>
<unk>
job-
seller..always
...
7
1-2
a++
3am3i
qaiser
raven09
liudmila
ksenka
venanzio
3d
dialux
asael
galland21
spoonlancer
timescales
subsiststudios
anoob
1st
ana-maria
2
3
yiga
chriistalrock
4
jeegy
100
5*
99designs
bossall691
charni
dark-o
ocelittle
99design
.extremely
micromanagement
firebase
redlogo
6th
work..
betiobca
qaulity
time-crunch
datafox
saimun
zunair
vtiger
sign.yra
laurra
anjul
bananodromo
katie_kat
a+++
umme
karlatut
susual
you..
after..
designs-
dmytro
hand-drawn/sketched/painted
re-designing
hourlies
2nd
bmdseoprovider
donmare
wintrygrey
a+++++
feedback/advice
prismonline
micro-alloying
time.y
lemoor
luthfun
eyecatching
reesee
vinith
ps_design
accomodating
creativeinfoway
fritzr
rosko
olgo
ginder
07
2018
02
andrewzz
exceded
gourav
experioence
dependably
maeastro
torvs
dreammaster
..
5
20139gph
thanks..
propositive
severak
expreience
brthr-ed
lucadia
krlegend
professionist
4th
-yes
sadafs
22
ateendras
standad
rziko1
re-hiring
igreg
12
shafayat
esolz
zenefasions


mahipal
sujit
turn-aound
kshitij
jcrbahia
m.c.plus.design
ubderstanding
efficeintly
effectuating
hermeneutic-
ardieaquino
kawalfram
mql4
grigoris
geo/mapping
virtuadmin
oroduct
enoyed
ambioned
deliveries..
designconceptz
prolv
cosultative
shebaz
loresil
vectorizing
freelancer..knowledgeable
expected..highly
clerero
mohinder
adilu
jashwant
navdeep
ksenya
.has
ueual
technofox
professionality
montz
shifullah
-smashing
timmings
angular/html/css
metrine
overdelivered
designer-
ηe
shahabaj
freelancer1
superbram
lauris79
wordmark
gigi39
listended
helpfule
alpesh
brokering
knoweldgeful
transmute
solicitous
mungunshagai
sudheer
profissionalism
mirkoandricdesign
pencil-tailored
romantka
studio..you
ಠ‿↼
ahmer
nasimoni
inforgraphics
commitemnt
work/designs
pleasure..fast
extrememly
lulia
deadlines/
2019
defienetly
servcie
pomptly
fanstatic
work.very
brief/requests
muditha
12,000
prasanth
jacekkadiziolka
sotiris
to/fro
louise-amy
ratnesh
copyediting
8k
cmyk13
design/needs
there´s
recordeod
client-u

## Neural Network Class

NOTE: Data loader is defined as:
- tuple: (tokens, flagged_index, problematic)

In [51]:
class neuralNetBow_glove(nn.Module):
    """
    BagOfWords classification model
    """
    # NOTE: we can't use linear layer until we take weighted average, otherwise it will
    # remember certain positions incorrectly (ie, 4th word has bigger weights vs 7th word)
    def __init__(self, embedding_matrix, upweight=10):
        super(neuralNetBow_glove, self).__init__()
        vocab_size = embedding_matrix.shape[0]
        embed_size = embedding_matrix.shape[1]
        
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        
        self.embed = nn.Embedding(vocab_size, embed_size, padding_idx=2)
        
        self.embed.weight = nn.Parameter(torch.tensor(embedding_matrix,
                                                          dtype=torch.float32))
        self.embed.weight.requires_grad = False
        
        
        self.upweight = upweight
    
    def forward(self, tokens, flagged_index):
        batch_size, num_tokens = tokens.shape
        embedding = self.embed(tokens)
#         print(embedding.shape) # below assumes "batch_size x num_tokens x Emb_dim" (VERIFY)
        
        # upweight by flagged_index
#         print(type(embedding))
        embedding[torch.LongTensor(range(batch_size)),flagged_index.type(torch.LongTensor),:] *= self.upweight
        
        # average across embeddings
        embedding_ave = embedding.sum(1) / (num_tokens + self.upweight - 1)
        
        return embedding_ave

### Clustering Stuff (un-tailored)

In [52]:
class KMeansCriterion(nn.Module):
    
    def __init__(self, lmbda):
        super().__init__()
        self.lmbda = lmbda
    
    def forward(self, embeddings, centroids):
        distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
        cluster_distances, cluster_assignments = distances.min(1)
        loss = self.lmbda * cluster_distances.sum()
        return loss, cluster_assignments

In [53]:
def centroid_init(k, d, dataloader, model, current_device):
    ## Here we ideally don't want to do randomized/zero initialization
    centroid_sums = torch.zeros(k, d).to(current_device)
    centroid_counts = torch.zeros(k).to(current_device)
    for (tokens, labels, flagged_indices) in dataloader:
        # cluster_assignments = torch.LongTensor(tokens.size(0)).random_(k)
        cluster_assignments = labels.to(current_device)
        
        model.eval()
        sentence_embed = model(tokens.to(current_device),flagged_indices.to(current_device))
    
        update_clusters(centroid_sums, centroid_counts,
                        cluster_assignments, sentence_embed.to(current_device))
    
    centroid_means = centroid_sums / centroid_counts[:, None].to(current_device)
    return centroid_means.clone()

def update_clusters(centroid_sums, centroid_counts,
                    cluster_assignments, embeddings):
    k = centroid_sums.size(0)

    centroid_sums.index_add_(0, cluster_assignments, embeddings)
    bin_counts = torch.bincount(cluster_assignments,minlength=k).type(torch.FloatTensor).to(current_device)
    centroid_counts.add_(bin_counts)
    
    #np_cluster_assignments = cluster_assignments.to('cpu')
    #np_counts = np.bincount(np_cluster_assignments.data.numpy(), minlength=k)
    #centroid_counts.add_(torch.FloatTensor(np_counts))

### Training Function (un-tailored, needs alterations)

In [54]:
def train_model(model, centroids, criterion, optimizer, train_loader, valid_loader, num_epochs=10, path_to_save=None, print_every = 1000):

    train_losses=[]
    val_losses=[]
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    for epoch in range(num_epochs):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.train()
        k, d = centroids.size()
        centroid_sums = torch.zeros_like(centroids).to(current_device)
        centroid_counts = torch.zeros(k).to(current_device)
        total_epoch_loss = 0

        # run one epoch of gradient descent on autoencoders wrt centroids
        for i, (tokens, labels, flagged_indices) in tqdm(enumerate(train_loader)):
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)

            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids.detach())

            # run update step
            optimizer.zero_grad()
#             cluster_loss.backward()
            optimizer.step()
            
            #Add loss to the epoch loss
            total_epoch_loss += cluster_loss.data

            # store centroid sums and counts in memory for later centering
            update_clusters(centroid_sums, centroid_counts,
                            cluster_assignments, sentence_embed)

            if i % print_every == 0:
                losses = cluster_loss.data/len(tokens)
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= len(train_loader.dataset)
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # update centroids based on assignments from autoencoders
        centroids = centroid_sums / (centroid_counts[:, None] + 1).to(current_device)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)
            
            #Add loss to the validation loss
            total_validation_loss += cluster_loss.data

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            opts = {"vocab_size":model.vocab_size, "embed_size": model.embed_size}
            torch.save(model.state_dict(), path_to_save+'model_dict.pt')
            torch.save(centroids, path_to_save+'centroids')
            torch.save(train_losses, path_to_save+'train_losses')
            torch.save(val_losses, path_to_save+'val_losses')
            torch.save(opts, path_to_save+'opts')
            
        
    return model, centroids, train_losses, val_losses

In [55]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

model = neuralNetBow_glove(glove_embedding_index).to(current_device)

In [56]:
# model = neuralNetBow(opts['vocab_size'], opts['emb_dim'])
centroids = centroid_init(2, 200,train_loader_labelled, model, current_device)
criterion = KMeansCriterion(1).to(current_device)
optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)

In [57]:
centroids

tensor([[ 6.8292e-02,  1.0521e-02, -1.1470e-02,  7.2092e-02, -1.8306e-02,
          1.0145e-01,  2.8836e-01,  9.0186e-03, -8.0530e-02, -1.0073e-01,
         -8.2945e-02, -8.3735e-02, -3.7348e-01, -6.0185e-02,  5.7898e-04,
          6.9175e-02,  1.6187e-02,  7.5986e-03, -6.3130e-02, -2.7320e-02,
         -5.9389e-02,  8.5293e-03, -4.1944e-02,  4.8728e-02, -7.6512e-02,
          4.8027e-01, -3.7244e-02,  1.0092e-01,  1.3747e-01,  1.0604e-02,
         -2.9348e-02, -9.7453e-02, -4.3732e-02,  1.7378e-02,  1.1165e-01,
          9.2251e-02,  2.6641e-02, -3.0175e-02,  7.8488e-03, -1.6710e-02,
          2.3404e-01, -1.3387e-02,  4.8180e-02,  4.4078e-02, -1.7893e-02,
         -2.1284e-03,  7.7306e-02,  2.4613e-02, -6.8325e-02,  9.8534e-03,
          3.3565e-03,  1.9454e-02, -1.2342e-01, -2.6541e-03,  9.3597e-02,
          6.1619e-03,  9.5398e-03, -4.9843e-02, -1.7416e-02, -2.6598e-02,
         -1.4866e-02,  4.4059e-03, -9.9971e-02, -4.5498e-02,  5.4118e-02,
          7.1149e-02, -8.5065e-02, -3.

In [58]:
current_device

'cpu'

In [59]:
review_dict.get_id("the")

41

In [60]:
torch.tensor([41])

tensor([41])

In [61]:
model.embed(torch.tensor([41]).to(current_device))

tensor([[ 4.9341e-01,  3.5693e-01,  6.6064e-01, -3.2990e-02,  2.4988e-01,
          2.5928e-01, -2.7176e-02,  6.8420e-02, -2.9053e-01, -4.5703e-01,
         -7.7942e-02,  3.2520e-01, -1.4854e+00, -6.7444e-02, -1.7029e-01,
         -9.2926e-03,  3.4619e-01, -1.1574e-02,  3.7964e-02,  4.5605e-01,
          8.0505e-02,  1.5308e-01, -1.5308e-01, -1.8811e-01, -1.8201e-01,
          8.7256e-01,  3.9795e-01,  4.0991e-01,  4.4971e-01, -1.9646e-03,
         -4.1138e-02, -4.7882e-02, -2.4048e-01, -8.6853e-02,  1.4183e-02,
         -2.3755e-01,  2.5171e-01,  2.8540e-01,  4.4507e-01, -4.9634e-01,
         -1.2708e-01, -1.7480e-01,  8.2214e-02,  4.5410e-02,  5.1709e-01,
          3.4546e-02, -8.5815e-02, -3.4912e-01,  5.2197e-01, -3.9502e-01,
          6.4148e-02, -4.2017e-01, -1.5942e-01,  1.8286e-01, -5.7892e-02,
         -1.9180e-02, -4.4556e-01,  3.1543e-01, -1.6101e-01, -9.2163e-02,
         -2.4963e-01, -1.3895e-03, -4.2651e-01, -1.7932e-01,  8.1665e-02,
          1.8323e-01, -3.2056e-01, -1.

In [69]:
path = os.getcwd()
model_folder= 'baseline_frozen_glove/'
model_dir = path + '/models/' + model_folder

In [71]:
baseline_model, baseline_centroids, baseline_train_losses, baseline_val_losses = train_model(model, centroids, criterion, optimizer, train_loader, val_loader, num_epochs=10, path_to_save=model_dir)



2019-11-17 20:13:05.978552 | Epoch 0


Average training loss at batch  0 : 2.512
Average training loss at batch  1000 : 1.923
Average training loss at batch  2000 : 2.155
Average training loss at batch  3000 : 1.843

Average training loss after epoch  0 : 1.996
Average validation loss after epoch  0 : 1.599
2019-11-17 20:13:08.835436 | Epoch 1


Average training loss at batch  0 : 1.940
Average training loss at batch  1000 : 1.906
Average training loss at batch  2000 : 1.520
Average training loss at batch  3000 : 1.488

Average training loss after epoch  1 : 1.753
Average validation loss after epoch  1 : 1.587
2019-11-17 20:13:11.524475 | Epoch 2


Average training loss at batch  0 : 2.083
Average training loss at batch  1000 : 2.075
Average training loss at batch  2000 : 1.481
Average training loss at batch  3000 : 1.457

Average training loss after epoch  2 : 1.679
Average validation loss after epoch  2 : 1.583
2019-11-17 20:13:14.236976 | Epoch 3


Average training loss at batch  0 : 1.438
Average training loss at batch  1000 : 1.816
Average training loss at batch  2000 : 1.892
Average training loss at batch  3000 : 1.768

Average training loss after epoch  3 : 1.653
Average validation loss after epoch  3 : 1.576
2019-11-17 20:13:16.857263 | Epoch 4


...

Average training loss at batch  0 : 1.524
Average training loss at batch  1000 : 1.358
Average training loss at batch  2000 : 1.580
Average training loss at batch  3000 : 1.687

Average training loss after epoch  5 : 1.638
Average validation loss after epoch  5 : 1.568
2019-11-17 20:13:22.104621 | Epoch 6


Average training loss at batch  0 : 1.566
Average training loss at batch  1000 : 1.869
Average training loss at batch  2000 : 2.058
Average training loss at batch  3000 : 1.610

Average training loss after epoch  6 : 1.636
Average validation loss after epoch  6 : 1.566
2019-11-17 20:13:24.782068 | Epoch 7


Average training loss at batch  0 : 1.386
Average training loss at batch  1000 : 1.831
Average training loss at batch  2000 : 1.638
Average training loss at batch  3000 : 1.501

Average training loss after epoch  7 : 1.636
Average validation loss after epoch  7 : 1.565
2019-11-17 20:13:27.420029 | Epoch 8


Average training loss at batch  0 : 1.890
Average training loss at batch  1000 : 1.470
Average training loss at batch  2000 : 1.628
Average training loss at batch  3000 : 1.386

Average training loss after epoch  8 : 1.635
Average validation loss after epoch  8 : 1.564
2019-11-17 20:13:30.088556 | Epoch 9


Average training loss at batch  0 : 1.449
Average training loss at batch  1000 : 1.458
Average training loss at batch  2000 : 1.684
Average training loss at batch  3000 : 1.646

Average training loss after epoch  9 : 1.635
Average validation loss after epoch  9 : 1.564


In [72]:
# #Only needed for Kaggle

# from IPython.display import FileLink, FileLinks 
# FileLinks('.') #lists all downloadable files on server